# Imports


In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-geometric


     |████████████████████████████████| 7.9 MB 5.2 MB/s 
     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 370 kB 5.5 MB/s 
     |████████████████████████████████| 482 kB 38.3 MB/s 
     |████████████████████████████████| 41 kB 542 kB/s 


In [ ]:
import sys
import os

import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from scipy.sparse import identity
from torch_geometric.nn import GCNConv
from torch_geometric.utils import from_scipy_sparse_matrix
import torch.utils.data
from torch.utils.tensorboard import SummaryWriter

if not torch.cuda.is_available():
    raise Exception("You should enable GPU runtime")
device = torch.device("cuda")

from google.colab import drive
drive.mount('/content/gdrive/')
sys.path.append('/content/gdrive/MyDrive/project/AIDL_2022_drug_recomendation/AIDL_project')

#importing customized libraries
import data_import
import metrics
import FM_model
import FM_GCN_model
import train

# Reading dataset

In [ ]:
dataset = data_import.Dataset(type_data="protein_disease", num_negatives_test = 2616-2)

# Loading tensorflow

In [ ]:
logs_base_dir = "runs"
os.makedirs(logs_base_dir, exist_ok=True)

%load_ext tensorboard 


# Common parameters
topk will be setted to 10, and the criterion will be BCEwithlogitsloss

In [ ]:
topk = 10
criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')

# FactorizationMachineModel

In [ ]:
from torch.utils.data import DataLoader

data_loader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=0)

model = FM_model.FactorizationMachineModel(dataset.field_dims[-1], 32).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

tb_fm = SummaryWriter(log_dir=f'{logs_base_dir}/{logs_base_dir}_FM/')
tb_fm = train.train_epochs(model, optimizer, data_loader, dataset, criterion, device, topk, tb_fm, epochs=150)




# FM with GCN

In [ ]:
X = FM_GCN_model.sparse_mx_to_torch_sparse_tensor(identity(dataset.train_mat.shape[0]))
edge_idx, edge_attr = from_scipy_sparse_matrix(dataset.train_mat)


In [ ]:
model_gcn = FM_GCN_model.FactorizationMachineModel_withGCN(dataset.field_dims[-1],
                                              64,
                                              X.to(device),
                                              edge_idx.to(device),
                                              ).to(device)

optimizer = torch.optim.Adam(params=model_gcn.parameters(), lr=0.001)

tb_gcn = SummaryWriter(log_dir=f'{logs_base_dir}/{logs_base_dir}_GCN/')

tb_gcn = train.train_epochs(model_gcn, optimizer, data_loader, dataset, criterion, device, topk, tb_gcn, epochs=150)

# GCN ATT

In [ ]:
model_gcn_att = FM_GCN_model.FactorizationMachineModel_withGCN(dataset.field_dims[-1],
                                                  64,
                                                  X.to(device),
                                                  edge_idx.to(device),
                                                  attention=True
                                                  ).to(device)

criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
optimizer = torch.optim.Adam(params=model_gcn_att.parameters(), lr=0.001)

tb_gcn_attention = SummaryWriter(log_dir=f'{logs_base_dir}/{logs_base_dir}_GCN_att/')

tb_gcn_attention = train.train_epochs(model_gcn_att, optimizer, data_loader, dataset, criterion, device, topk, tb_gcn_attention, epochs=150)

# Tensorboard

In [ ]:
%tensorboard --logdir runs